In [8]:
import os
import requests
import json
import gzip
import pandas as pd

In [ ]:
companies = ['LLY', 'APPL', 'BEP']
params = {'function': 'TIME_SERIES_INTRADAY',
          'symbol': 'LLY',
          'interval': '1min',
          'month': '2023-01',
          'outputsize': 'full',
          'extended_hours': 'false',
          'apikey': os.environ["API_KEY"]
          }

In [4]:
yyyy = '2025'

In [5]:
mm=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

In [10]:
mm=['01', '02', '03', '04', '05', '06', '07', '08', '09']

In [16]:
for m in mm:
    params['month']=yyyy+"-"+m
    url = "https://www.alphavantage.co/query?"
    for key, val in params.items():
        url+=key
        url+='='
        url+=val
        url+='&'
    url = url[:-1]
    r = requests.get(url)
    data = r.json()
    print(m, len(data['Time Series (1min)']))
    df = pd.DataFrame.from_dict(data['Time Series (1min)'], orient='index')
    df = df.sort_index()
    # print(df)
    df.to_csv(params['symbol'] + params['month'] + '.csv.gz', compression='gzip')



01 7741
02 7362
03 8132
04 8147
05 8157
06 7686
07 8397
08 8156
09 8110


In [17]:
data = pd.read_csv("LLY2025-01.csv.gz")
data

,Unnamed: 0,1. open,2. high,3. low,4. close,5. volume
0,2025-01-02 09:30:00,776.0055,777.9043,774.5640,774.7617,63599
1,2025-01-02 09:31:00,775.8265,780.6381,775.8265,780.6282,13154
2,2025-01-02 09:32:00,780.6381,781.1054,778.7493,780.3896,8929
3,2025-01-02 09:33:00,780.0914,780.0914,779.4949,779.4949,3560
4,2025-01-02 09:34:00,779.2712,779.5247,777.7552,778.3020,10824
...,...,...,...,...,...,...
7736,2025-01-31 15:55:00,807.7380,808.2509,806.5650,807.1018,25666
7737,2025-01-31 15:56:00,807.2509,807.7380,806.4357,806.6047,27667
7738,2025-01-31 15:57:00,807.1515,807.6088,806.5749,806.9725,40463
7739,2025-01-31 15:58:00,806.9924,807.2211,806.2965,806.4059,39389
